In [1]:
%load_ext autoreload
%autoreload 2
from covid_constants_and_util import *
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import json
import datetime
import copy
import geopandas as gpd
import dask
import helper_methods_for_aggregate_data_analysis as helper
import h5py
import re

JUST_TESTING = False

Setting numpy cores to 1
Running code on rambo.stanford.edu; at Stanford=True


ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
# make sure we don't append onto existing files [for H5 files]. 
assert not os.path.exists(os.path.join(helper.ANNOTATED_H5_DATA_DIR, helper.CHUNK_FILENAME))

In [3]:
# Write out dataframe of Census data for use in subsequent analysis. 
helper.write_out_acs_5_year_data()

ACS_2017_5YR_BG.gdb
['X00_COUNTS', 'X01_AGE_AND_SEX', 'X02_RACE', 'X03_HISPANIC_OR_LATINO_ORIGIN', 'X07_MIGRATION', 'X08_COMMUTING', 'X09_CHILDREN_HOUSEHOLD_RELATIONSHIP', 'X11_HOUSEHOLD_FAMILY_SUBFAMILIES', 'X12_MARITAL_STATUS_AND_HISTORY', 'X14_SCHOOL_ENROLLMENT', 'X15_EDUCATIONAL_ATTAINMENT', 'X16_LANGUAGE_SPOKEN_AT_HOME', 'X17_POVERTY', 'X19_INCOME', 'X20_EARNINGS', 'X21_VETERAN_STATUS', 'X22_FOOD_STAMPS', 'X23_EMPLOYMENT_STATUS', 'X24_INDUSTRY_OCCUPATION', 'X25_HOUSING_CHARACTERISTICS', 'X27_HEALTH_INSURANCE', 'X99_IMPUTATION', 'BG_METADATA_2017', 'ACS_2017_5YR_BG']
Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'Shape_Length', 'Shape_Area', 'GEOID_Data', 'geometry'],
      dtype='object')
Length of demographic data: 220333; geographic data 220740; 220333 GEOIDs in both
72    2594
Name: STATEFP, dtype: int64
2594/220333 Census Block Groups in total removed
annotating with race


In [4]:
# read in individual dataframes for monthly and weekly data [raw SafeGraph data].
dask.config.set(pool=ThreadPool(20))

all_monthly_dfs = []
all_weekly_dfs = []   

for week_string in helper.ALL_WEEKLY_STRINGS:
    all_weekly_dfs.append(helper.load_patterns_data(week_string=week_string, just_testing=JUST_TESTING))
    
for month, year in [
             (1, 2019),(2, 2019),(3, 2019),(4, 2019),(5, 2019),(6, 2019),(7, 2019),(8, 2019),(9, 2019),(10, 2019),(11, 2019),(12, 2019),
             (1, 2020),(2, 2020)][::-1]:
    # Note ::-1: we load most recent files first so we will take their places info if it is available.
    all_monthly_dfs.append(helper.load_patterns_data(month=month, year=year, just_testing=JUST_TESTING))
    


Offset from GMT value counts
05:00                1801254
06:00                1184004
08:00                 642032
07:00                 266714
10:00                  16607
09:00                   8822
04:00                   4682
01T00:00:00+10:00        119
11:00                      6
Name: offset_from_gmt, dtype: int64
Prior to dropping rows with no visits by day, 3924240 rows
99.230% of rows have timezones that we spike-correct for.
3924240 rows loaded for week 2020-03-01
Offset from GMT value counts
05:00                1791060
06:00                1177173
08:00                 637855
07:00                 264818
10:00                  16463
09:00                   8656
04:00                   4651
08T00:00:00+10:00        119
11:00                      6
Name: offset_from_gmt, dtype: int64
Prior to dropping rows with no visits by day, 3900801 rows
99.234% of rows have timezones that we spike-correct for.
3900801 rows loaded for week 2020-03-08
Offset from GMT value counts
04:00

In [5]:
# Merge monthly DFs into a single dataframe. Each row is one POI. 
base = all_monthly_dfs[0]
core = all_monthly_dfs[1].columns.intersection(base.columns).to_list()
for i, df in enumerate(all_monthly_dfs[1:]):
    print(i)
    # merge all new places into base so that core info is not nan for new sgids
    new_places = df.loc[df.index.difference(base.index)][core]
    base = pd.concat([base, new_places], join='outer', sort=False)
    # can now left merge in the df because all sgids will be in base
    cols_to_use = df.columns.difference(base.columns).to_list()
    base =  pd.merge(base, df[cols_to_use], left_index=True, right_index=True, how='left')


0
1
2
3
4
5
6
7
8
9
10
11
12


In [6]:
# Merge in weekly dataframes. Just merge on SafeGraph ID, left merge. 
# This means that our final POI set is those that have both monthly and weekly data. 
# at the end of this cell we will have a single dataframe. 

for i, weekly_df in enumerate(all_weekly_dfs):
    print("\n\n********Weekly dataframe %i/%i" % (i + 1, len(all_weekly_dfs)))
    assert len(base.columns.intersection(weekly_df.columns)) == 0
    
    ids_in_weekly_but_not_monthly = set(weekly_df.index) - set(base.index)
    print("Warning: %i/%i POIs in weekly but not monthly data; dropping these" % (len(ids_in_weekly_but_not_monthly), 
                                                                  len(df)))
    base = pd.merge(base, weekly_df, how='left', left_index=True, right_index=True, validate='one_to_one')
    print("Missing data for weekly columns")
    print(pd.isnull(base[weekly_df.columns]).mean())



********Weekly dataframe 1/9
Missing data for weekly columns
2020-03-01.visitor_home_cbgs            0.141896
2020-03-01.visitor_country_of_origin    0.147981
2020-03-01.distance_from_home           0.335333
2020-03-01.median_dwell                 0.141896
2020-03-01.bucketed_dwell_times         0.141896
2020.3.1                                0.141896
2020.3.2                                0.141896
2020.3.3                                0.141896
2020.3.4                                0.141896
2020.3.5                                0.141896
2020.3.6                                0.141896
2020.3.7                                0.141896
hourly_visits_2020.3.1.0                0.141896
hourly_visits_2020.3.1.1                0.141896
hourly_visits_2020.3.1.2                0.141896
hourly_visits_2020.3.1.3                0.141896
hourly_visits_2020.3.1.4                0.141896
hourly_visits_2020.3.1.5                0.141896
hourly_visits_2020.3.1.6                0.141896
hourly

In [7]:
# sanity check: how much do weekly visits change if we drop parent IDs. 
parent_ids = set(base['parent_safegraph_place_id'].dropna())
first_week_of_march_cols = ['hourly_visits_2020.3.%i.%i' % (i, j) for i in range(1, 8) for j in range(24)]
total_daily_child_visits = base.loc[~pd.isnull(base['parent_safegraph_place_id']), first_week_of_march_cols].dropna().values.sum()
total_daily_parent_visits = base.loc[base.index.map(lambda x:x in parent_ids), first_week_of_march_cols].dropna().values.sum()
total_daily_visits = base[first_week_of_march_cols].dropna().values.sum()
print("Total daily child visits are fraction %2.3f of total visits; parent visits are %2.3f; dropping parent visits" 
      % (total_daily_child_visits/total_daily_visits, total_daily_parent_visits/total_daily_visits))

# Drop parents to avoid double-counting visits. 
base = base.loc[base.index.map(lambda x:x not in parent_ids)]

Total daily child visits are fraction 0.125 of total visits; parent visits are 0.143; dropping parent visits


In [8]:
# annotate with demographic info and save dataframe. Dataframe is saved in h5py format, separated into chunks. 

annotated = base.sample(frac=1) # shuffle so rows are in random order [in case we want to prototype on subset].
annotated = helper.annotate_with_demographic_info_and_write_out_in_chunks(annotated, just_testing=JUST_TESTING)

Prior to merging with safegraph areas, 4379259 rows
After merging with areas, 4068665 rows
Mapping SafeGraph POIs to demographic info, including race and income.
ACS_2017_5YR_BG.gdb
['X00_COUNTS', 'X01_AGE_AND_SEX', 'X02_RACE', 'X03_HISPANIC_OR_LATINO_ORIGIN', 'X07_MIGRATION', 'X08_COMMUTING', 'X09_CHILDREN_HOUSEHOLD_RELATIONSHIP', 'X11_HOUSEHOLD_FAMILY_SUBFAMILIES', 'X12_MARITAL_STATUS_AND_HISTORY', 'X14_SCHOOL_ENROLLMENT', 'X15_EDUCATIONAL_ATTAINMENT', 'X16_LANGUAGE_SPOKEN_AT_HOME', 'X17_POVERTY', 'X19_INCOME', 'X20_EARNINGS', 'X21_VETERAN_STATUS', 'X22_FOOD_STAMPS', 'X23_EMPLOYMENT_STATUS', 'X24_INDUSTRY_OCCUPATION', 'X25_HOUSING_CHARACTERISTICS', 'X27_HEALTH_INSURANCE', 'X99_IMPUTATION', 'BG_METADATA_2017', 'ACS_2017_5YR_BG']
Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'Shape_Length', 'Shape_Area', 'GEOID_Data', 'geometry'],
      dtype='object')
Length of demographic data: 2

/afs/cs.stanford.edu/u/emmap1/safegraph/files_for_public_covid_repo/helper_methods_for_aggregate_data_analysis.py:641: RuntimeWarning: invalid value encountered in greater
  cbgs_with_ratio_above_one = cbgs_with_ratio_above_one | (combined_data['ratio'].values > 1)


2017.2: 13525824 devices read from 220024 rows
Number of unique Census blocks: 219712; unique blocks 220024: WARNING: DROPPING NON-UNIQUE ROWS
Missing data for 824 rows; filling with zeros
Ratio of SafeGraph count to Census count
count    219214.000000
mean          0.046712
std           0.527702
min           0.000000
25%           0.026720
50%           0.035983
75%           0.049074
90%           0.067815
99%           0.156075
99.9%         0.566156
max         144.000000
Name: ratio, dtype: float64
Correlation between SafeGraph and Census counts: 0.699

*************
2017.3: 15637522 devices read from 220135 rows
Number of unique Census blocks: 219747; unique blocks 220135: WARNING: DROPPING NON-UNIQUE ROWS
Missing data for 802 rows; filling with zeros
Ratio of SafeGraph count to Census count
count    219214.000000
mean          0.054975
std           0.695775
min           0.000000
25%           0.030977
50%           0.041511
75%           0.056691
90%           0.078981
99%  

/afs/cs.stanford.edu/u/emmap1/safegraph/files_for_public_covid_repo/helper_methods_for_aggregate_data_analysis.py:448: RuntimeWarning: divide by zero encountered in true_divide
  cbg_coverage = num_devices / total_population
/afs/cs.stanford.edu/u/emmap1/safegraph/files_for_public_covid_repo/helper_methods_for_aggregate_data_analysis.py:448: RuntimeWarning: invalid value encountered in true_divide
  cbg_coverage = num_devices / total_population
/afs/cs.stanford.edu/u/emmap1/safegraph/files_for_public_covid_repo/helper_methods_for_aggregate_data_analysis.py:510: RuntimeWarning: divide by zero encountered in double_scalars
  upweighting_factor = 1 / cbg_coverage[cbg]  # need to invert coverage


Finished adjusting home CBG counts for 2020.2 [time=1.958s] had to fill in or clip coverage for 0.314000% of rows; in those cases used median coverage 0.115
Filling 2020.1.visitor_home_cbgs with Counter objects
Finished adjusting home CBG counts for 2020.1 [time=1.851s] had to fill in or clip coverage for 0.370000% of rows; in those cases used median coverage 0.122
Filling 2019.12.visitor_home_cbgs with Counter objects
Finished adjusting home CBG counts for 2019.12 [time=1.859s] had to fill in or clip coverage for 0.335000% of rows; in those cases used median coverage 0.102
Filling 2019.11.visitor_home_cbgs with Counter objects
Finished adjusting home CBG counts for 2019.11 [time=1.721s] had to fill in or clip coverage for 0.218000% of rows; in those cases used median coverage 0.088
Filling 2019.10.visitor_home_cbgs with Counter objects
Finished adjusting home CBG counts for 2019.10 [time=1.559s] had to fill in or clip coverage for 0.181000% of rows; in those cases used median coverage

/dfs/scratch1/safegraph_homes/safegraph_env_v3/safegraph_env_v3/lib/python3.7/site-packages/pandas/core/generic.py:2368: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['parent_safegraph_place_id', 'location_name', 'top_category', 'sub_category', 'city', 'region', 'polygon_wkt', 'polygon_class', '2020.2.visitor_home_cbgs', '2020.2.visitor_country_of_origin', '2020.2.bucketed_dwell_times', '2020.1.bucketed_dwell_times', '2020.1.visitor_country_of_origin', '2020.1.visitor_home_cbgs', '2019.12.bucketed_dwell_times', '2019.12.visitor_country_of_origin', '2019.12.visitor_home_cbgs', '2019.11.bucketed_dwell_times', '2019.11.visitor_country_of_origin', '2019.11.visitor_home_cbgs', '2019.10.bucketed_dwell_times', '2019.10.visitor_country_of_origin', '2019.10.visitor_home_cbgs', '2019.9.bucketed_dwell_times', '2019.9.visitor_country_of_origin', '2019.9.visitor_home_cbg

******************Annotating chunk 1
Doing spatial join on points with indices from 0-100000
Cannot match to a CBG for a fraction 0.001 of points
Total query time is 54.129
Aggregating data from: ['2020.2.visitor_home_cbgs', '2020.1.visitor_home_cbgs', '2019.12.visitor_home_cbgs', '2019.11.visitor_home_cbgs', '2019.10.visitor_home_cbgs', '2019.9.visitor_home_cbgs', '2019.8.visitor_home_cbgs', '2019.7.visitor_home_cbgs', '2019.6.visitor_home_cbgs', '2019.5.visitor_home_cbgs', '2019.4.visitor_home_cbgs', '2019.3.visitor_home_cbgs', '2019.2.visitor_home_cbgs', '2019.1.visitor_home_cbgs']
Filling 2020.2.visitor_home_cbgs with Counter objects
Finished adjusting home CBG counts for 2020.2 [time=1.994s] had to fill in or clip coverage for 0.344000% of rows; in those cases used median coverage 0.115
Filling 2020.1.visitor_home_cbgs with Counter objects
Finished adjusting home CBG counts for 2020.1 [time=2.225s] had to fill in or clip coverage for 0.397000% of rows; in those cases used median c

In [9]:
# Stratify by MSA and write out outfiles.  
just_in_msas = annotated.loc[annotated['poi_lat_lon_Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area']
assert pd.isnull(just_in_msas['poi_lat_lon_CBSA Title']).sum() == 0  # POIs in MSAs must have CBSA title
print("%i/%i POIs are in MSAs (%i MSAs total)" % (len(just_in_msas), 
                                                  len(annotated), 
                                                  len(set(just_in_msas['poi_lat_lon_CBSA Title']))))
grouped_by_msa = just_in_msas.groupby('poi_lat_lon_CBSA Title')
total_written_out = 0
for msa_name, small_d in grouped_by_msa:
    small_d = small_d.copy().sample(frac=1) # make sure rows in random order. 
    small_d.index = range(len(small_d))
    name_without_spaces = re.sub('[^0-9a-zA-Z]+', '_', msa_name)
    filename = os.path.join(helper.STRATIFIED_BY_AREA_DIR, '%s.csv' % name_without_spaces)
    for k in ['aggregated_cbg_population_adjusted_visitor_home_cbgs', 'aggregated_visitor_home_cbgs']:
        small_d[k] = small_d[k].map(lambda x:json.dumps(dict(x))) # cast to json so properly saved in CSV. 
    small_d.to_csv(filename)
    print("Wrote out dataframe with %i POIs to %s" % (len(small_d), '%s.csv' % name_without_spaces))
    total_written_out += 1
print("Total written out: %i" % total_written_out)

3410357/4068665 POIs are in MSAs (383 MSAs total)
Wrote out dataframe with 2592 POIs to Abilene_TX.csv
Wrote out dataframe with 8448 POIs to Akron_OH.csv
Wrote out dataframe with 2086 POIs to Albany_GA.csv
Wrote out dataframe with 1505 POIs to Albany_OR.csv
Wrote out dataframe with 10609 POIs to Albany_Schenectady_Troy_NY.csv
Wrote out dataframe with 10218 POIs to Albuquerque_NM.csv
Wrote out dataframe with 2134 POIs to Alexandria_LA.csv
Wrote out dataframe with 10327 POIs to Allentown_Bethlehem_Easton_PA_NJ.csv
Wrote out dataframe with 1780 POIs to Altoona_PA.csv
Wrote out dataframe with 3660 POIs to Amarillo_TX.csv
Wrote out dataframe with 1122 POIs to Ames_IA.csv
Wrote out dataframe with 5271 POIs to Anchorage_AK.csv
Wrote out dataframe with 4378 POIs to Ann_Arbor_MI.csv
Wrote out dataframe with 1634 POIs to Anniston_Oxford_Jacksonville_AL.csv
Wrote out dataframe with 3178 POIs to Appleton_WI.csv
Wrote out dataframe with 7217 POIs to Asheville_NC.csv
Wrote out dataframe with 2729 PO